# Alechemy Nodes Data Analysis

In [237]:
import requests
import json
import pandas as pd
from bs4 import BeautifulSoup
import time

In [238]:
# Get the wallet address
def decode_topic_address(topic_hex):
    return "0x" + topic_hex[-40:]

In [ ]:
# Scrape contract name from address on etherscan.io
def get_contract_name(address):
    url = f"https://etherscan.io/address/{address}"
    headers = {
        "User-Agent": "Mozilla/5.0"
    }

    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, "html.parser")

    title_text = soup.title.string.strip()

    contract_name = title_text.split("(")[0].strip()

    return contract_name


In [ ]:
from web3 import Web3

ALCHEMY_API_KEY = ""
alchemy_url = f"https://eth-mainnet.g.alchemy.com/v2/{ALCHEMY_API_KEY}"

w3 = Web3(Web3.HTTPProvider(alchemy_url))

print("Connected:", w3.is_connected())


Connected: True


In [241]:
# List of ERC-20 tokens

url = "https://tokens.uniswap.org"
response = requests.get(url)
token_data = response.json()

In [243]:
# Block Info

block_number = 22152102

payload = {
    "jsonrpc": "2.0",
    "id": 1,
    "method": "eth_getBlockByNumber",
    "params": [hex(block_number),False]
}

headers = {"Content-Type": "application/json"}

# Send request
response = requests.post(alchemy_url, headers=headers, json=payload)

# Parse the JSON response
block_info_data = response.json()

# Extract results (receipts from all transactions in the block)
block_info = block_info_data.get("result", [])

In [244]:
with open(f"block_{block_number}_info.json", "w") as file:
    json.dump(block_info_data, file, indent=2)

In [245]:
# Receipts in the block

payload = {
    "jsonrpc": "2.0",
    "id": 1,
    "method": "eth_getBlockReceipts",
    "params": [hex(latest_block)]
}

headers = {"Content-Type": "application/json"}

# Send request
response = requests.post(alchemy_url, headers=headers, json=payload)

# Parse the JSON response
receipts_data = response.json()

# Extract results (receipts from all transactions in the block)
receipts = receipts_data.get("result", [])

In [246]:
with open(f"block_{block_number}_receipts.json", "w") as file:
    json.dump(receipts_data, file, indent=2)

In [247]:
uncles_number = len(block_info['uncles'])
miner = get_contract_name(block_info['miner'])
gas_limit = int(block_info['gasLimit'],16)
gas_used = int(block_info['gasUsed'],16)
base_fee = int(block_info['baseFeePerGas'],16)
withdrawal_nb = len(block_info['withdrawals'])
gas_target = gas_limit/2
transactions_nb = len(block_info['transactions'])

# Total Miners reward: 

miners_reward = 0

for rec in receipts:
    rec_gas_used = int(rec['gasUsed'], 16)
    rec_effective_gas_price = int(rec['effectiveGasPrice'], 16)
    priority_fee = rec_effective_gas_price - base_fee  # All in wei
    reward = rec_gas_used * priority_fee
    miners_reward += reward

miners_reward = miners_reward/1000000000**2

In [ ]:
# Blocks data

print(f"Miner: {miner}")
print(f"Block number: {latest_block}")
print(f"Number of uncles: {uncles_number}") # After the merge uncle blocks were completely removed
print(f"Number of transactions: {transactions_nb}")
print(f"Gas Limit (wei): {gas_limit}")
print(f"Gas Used (wei): {gas_used}")
print(f"Gas Target (wei): {gas_target}")
print(f"Over Target (%): {((gas_used-gas_target)/gas_target)*100}%")
print(f"Base Fee per gas (wei): {base_fee}")
print(f"Burnt ETH in the block: {base_fee*gas_used/1000000000**2}")
print(f"Miners Reward (includes only fee reward) (ETH): {miners_reward}") # The miners reward includes only the fee reward and not consensus layer reward. To get consensus layer reward, see beaconcha.in
print(f"Number of validators withdrawls: {withdrawal_nb}")

Miner: titanbuilder.eth | Address 0x4838b106fce9647bdf1e7877bf73ce8b0bad5f97 | Etherscan
Block number: 22152102
Number of uncles: 0
Number of transactions: 255
Gas Limit (wei): 35964845
Gas Used (wei): 29450823
Gas Target (wei): 17982422.5
Over Target (%): 63.775614770479336%
Base Fee per gas (wei): 377861915
Burnt ETH in the block: 0.011128344377106044
Miners Reward (includes only fee reward): 0.05002913914257148
Number of validators withdrawls: 16


In [248]:
# Validators Withdrawals

withdrawals = []

for withdrawal in block_info['withdrawals']:
    withdrawals.append(
    {"Validator Index": int(withdrawal['validatorIndex'],16),
     "Wallet Address": withdrawal['address'],
     "Amount (ETH)": int(withdrawal['amount'],16)/1000000000
    }
    )
withdrawals_df=pd.DataFrame(withdrawals)

In [249]:
print(f"Validators full withdrawal: {(withdrawals_df['Amount (ETH)'] >= 32).sum()}")
withdrawals_df.head()

Validators full withdrawal: 0


,Validator Index,Wallet Address,Amount (ETH)
0,1390469,0x22d8048f3ccfbd505611b148d2f35d9aa1890a52,0.018918
1,1390470,0x22d8048f3ccfbd505611b148d2f35d9aa1890a52,0.018921
2,1390472,0x22d8048f3ccfbd505611b148d2f35d9aa1890a52,0.018929
3,1390473,0x22d8048f3ccfbd505611b148d2f35d9aa1890a52,0.018933
4,1390475,0x22d8048f3ccfbd505611b148d2f35d9aa1890a52,0.018905


We observe in the DataFrame that no validator withdrew the full 32 ETH, indicating that these are partial withdrawals. This suggests that the validators are still active and only withdrew the rewards earned above their original stake. Validators must maintain at least 32 ETH to continue validating, and dropping below that threshold would require them to exit. The ETH being withdrawn here represents staking rewards, not gas fees or tips.

In [250]:
#Pick first receipt for analysis

receipt = receipts[0]

In [251]:
# Verify maximal and minimal number of topics

temp = []

for log in receipt["logs"]:
    if log["topics"][0]==ERC_20_transfer:
        temp.append(len(log["topics"]))

print(f"Maximal number of topics is {max(temp)} and minimal is {min(temp)}")

Maximal number of topics is 3 and minimal is 3


In [252]:
# Get only logs with ERC-20 transfer events (i.e only events of received and sent)

ERC_20_transfer  = "0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef"

rows = []
for log in receipt["logs"]:
    if log["topics"][0]==ERC_20_transfer:
        # Token contract address (e.g. USDC)
        contract_address = w3.to_checksum_address(log['address'])

        # Minimal ERC-20 ABI
        erc20_abi = [
        {"name": "name", "outputs": [{"type": "string"}], "type": "function", "stateMutability": "view"},
        {"name": "symbol", "outputs": [{"type": "string"}], "type": "function", "stateMutability": "view"},
        {"name": "decimals", "outputs": [{"type": "uint8"}], "type": "function", "stateMutability": "view"}
        ]

        contract = w3.eth.contract(address=contract_address, abi=erc20_abi)

        # Call token functions
        name = contract.functions.name().call()
        symbol = contract.functions.symbol().call()
        decimals = contract.functions.decimals().call()

        log_from=get_contract_name(decode_topic_address(log['topics'][1]))
        time.sleep(2)
        log_to=get_contract_name(decode_topic_address(log['topics'][2]))
        rows.append(
        {"Token": symbol,
        "event_from": log_from,
        "event_to": log_to,
        "quantity": int(log['data'],16)/(10**decimals), 
        "removed": log['removed']
        }
        )

df = pd.DataFrame(rows)

In [ ]:
transaction_hash = receipt['transactionHash']
high_level_from = receipt['from']
high_level_to = receipt['to']
effective_gas_price = int(receipt['effectiveGasPrice'],16)
receipt_gas_used = int(receipt['gasUsed'],16)


In [254]:
print(f"Transaction hash: {transaction_hash}")
print(f"High level from address: {high_level_from}")
print(f"High level to address: {high_level_to}")
print(f"Base fee per gas (wei): {base_fee}")
print(f"Effective Gas Price (wei): {effective_gas_price}") # Includes base fee + priority tip
print(f"Gas used (units): {receipt_gas_used}")
print(f"Priority tip (ETH): {(effective_gas_price-base_fee)*receipt_gas_used/1000000000**2}")
print(f"Burnt ETH from the transaction (ETH): {base_fee*receipt_gas_used/1000000000**2}")
print(f"Total paid fees for transaction (ETH): {receipt_gas_used*effective_gas_price/1000000000**2}")
df.head()

Transaction hash: 0x0fb7e9061e8f7e16b43f66a5406d500b4f1bc6368e085e85684e15940db00264
High level from address: 0x5b43453fce04b92e190f391a83136bfbecedefd1
High level to address: 0xfbd4cdb413e45a52e2c8312f670e9ce67e794c37
Base fee per gas (wei): 377861915
Effective Gas Price (wei): 23431287160
Gas used (units): 1132121
Priority tip (ETH): 0.026099266841794644
Burnt ETH from the transaction (ETH): 0.000427785409071715
Total paid fees for transaction (ETH): 0.02652705225086636


,Token,event_from,event_to,quantity,removed
0,USDC,Uniswap V3: USDC 3,Contract Address 0xfbd4cdb413e45a52e2c8312f670...,125069.012081,False
1,WETH,Contract Address 0xfbd4cdb413e45a52e2c8312f670...,Uniswap V3: USDC 3,66.983648,False
2,USDT,Uniswap V3: USDT 3,Contract Address 0xfbd4cdb413e45a52e2c8312f670...,39158.152060,False
3,WETH,Contract Address 0xfbd4cdb413e45a52e2c8312f670...,Uniswap V3: USDT 3,20.964810,False
4,USDC,Uniswap V3: USDC 2,Contract Address 0xfbd4cdb413e45a52e2c8312f670...,10325.558660,False


In [255]:
df.to_excel('Transaction logs.xlsx', index=False)

More endpoints proposed by Alchemy:
* eth_getProof - verify wallet balance at given block
* eth_getTransactionCount - number of transactions sent from and address
* eth_getUncleByBlockHashAndInde
* eth_getUncleCountByBlockHash
* eth_maxPriorityFeePerGas
* eth_newFilter
* eth_getFilterChanges
* eth_getFilterLogs
* eth_newBlockFilter
* eth_newPendingTransactionFilter
* eth_uninstallFilter
* eth_getLogs
* eth_getBalance
* eth_feeHistory
* eth_estimateGas
* eth_getStorageAt
* Alchemy_getAssetTransfers
* alchemy_getTokenBalances
* Event Listeners